# Sandpile

In this notebook we are going to investigate the Per Bak sandpile model for self-organized criticality.

In [1]:
from plotly import tools
from plotly import offline as py
from plotly import graph_objs as go

py.init_notebook_mode(connected=True)

The two dimensional sandpile model from Bak, Tang and Wiesenfeld defines a state, $$\begin{align}
h_{i,j}\in\{0,1,\dots,h^*-1\} && \forall (i,j)\in\{0,1,\dots,N\}^2,
\end{align}$$
wherein $h^*$ is a critical height value. On each iteration one selects a random $(i,j)$ and increases the height, $$h_{i,j}\to h_{i,j}+1$$. If a lattice site exceeds the critical height $h_{i,j}\geq h^*$ one applies the update, $$\begin{align}
h_{i,j}&\to h_{i,j}-h^*\\
h_{i\pm1,j}&\to h_{i\pm1,j}+1\\
h_{i,j\pm1}&\to h_{i,j\pm1}+1
\end{align},$$
until $h_{i,j}<h^*$ for every $i,j$. The edges of the lattice are set to zero to ensure that no saturation occurs.

One way to think of the model is that of a growing sandpile. Assuming that each sand grain is of equal size we would find a natural lattice spacing. If a local sandpile grows over a critical value it will become unstable and spread to its neighbors. If one of the neighbors also becomes unstable their height will also get redistributed. This can yield a chain reaction. We refer to the number of these toppling events as avalanches. We are going to use them later on for scaling analysis.

In [9]:
def sandpile(hmax, xsize, ysize, N):
    a = np.zeros(N+1)
    h = np.zeros((N+1, xsize + 2, ysize + 2))
    h[0, 1:xsize + 1, 1:ysize + 1] = np.random.randint(0, hmax, (xsize, ysize))
    
    for n in range(N):
        h[n+1] = h[n]
        
        i = np.random.randint(1, xsize + 1)
        j = np.random.randint(1, ysize + 1)

        h[n+1, i, j] += 1
        
        while h[n+1].max() >= hmax:
            select = h[n+1] >= hmax
            
            # avalanches
            a[n+1] += select.sum()
            
            # toppling
            h[n+1][select] -= 4
            h[n+1][1:, :][select[:-1, :]] += hmax / 4
            h[n+1][:-1,:][select[1:, :]] += hmax / 4
            h[n+1][:, 1:][select[:, :-1]] += hmax / 4
            h[n+1][:, :-1][select[:, 1:]] += hmax / 4
            
            # clear edges
            h[n+1][0, :] = 0
            h[n+1][xsize + 1, :] = 0
            h[n+1][:, 0] = 0
            h[n+1][:, ysize + 1] = 0
            
    return a, h[:, 1:-1, 1:-1]

In [10]:
a, h = sandpile(4, 10, 10, 100)

steps = []

for i in range(h.shape[0]):
    step = dict(
        method = 'restyle',  
        args = ['visible', [False] * h.shape[0]],
    )
    step['args'][1][i] = True
    steps.append(step)

layout = go.Layout(
    title='Sandpile',
    height=600,
    width=700,
    xaxis=dict(zeroline=False, showline=False, showticklabels=False, showgrid=False),
    yaxis=dict(zeroline=False, showline=False, showticklabels=False, showgrid=False),
    sliders=[
        dict(steps=steps, active=h.shape[0] - 1, pad={"t": 50}),
    ],
)

figure = go.Figure([
    go.Heatmap(
        z=h[n],
        zmin=0,
        zmax=4,
        colorscale='YlGnBu',
        reversescale=True,
        visible = False,
        name = f'step {n}',
    )
    for n in range(h.shape[0])
], layout)

figure['data'][-1]['visible'] = True

py.iplot(figure)

In [20]:
a1, h1 = sandpile(4, 10, 10, 10000)
a2, h2 = sandpile(4, 100, 100, 10000)
a3, h3 = sandpile(4, 1000, 1000, 10000)

In [83]:
aval1, count1 = np.unique(a1, return_counts=True)
aval2, count2 = np.unique(a2, return_counts=True)
aval3, count3 = np.unique(a3, return_counts=True)

# omit no avalanche events
count1 = count1[aval1 > 0]
count2 = count2[aval2 > 0]
count3 = count3[aval3 > 0]
aval1 = aval1[aval1 > 0]
aval2 = aval2[aval2 > 0]
aval3 = aval3[aval3 > 0]

# frequency to probability
prop1 = count1 / count1.max()
prop2 = count2 / count2.max()
prop3 = count3 / count3.max()

layout = go.Layout(
    title='Scaling',
    xaxis=dict(title='Avalanches', type='log'),
    yaxis=dict(title='Probability', type='log'),
)

figure = go.Figure([
    go.Scatter(x=aval1, y=prop1, mode='markers', name=f'{h1.shape[1]} x {h1.shape[2]}'),
    go.Scatter(x=aval2, y=prop2, mode='markers', name=f'{h2.shape[1]} x {h2.shape[2]}'),
    go.Scatter(x=aval3, y=prop3, mode='markers', name=f'{h3.shape[1]} x {h3.shape[2]}'),
], layout)

py.iplot(figure)

In [84]:
logprop1 = np.log10(prop1)
logprop2 = np.log10(prop2)
logprop3 = np.log10(prop3)

logaval1 = np.log10(aval1)
logaval2 = np.log10(aval2)
logaval3 = np.log10(aval3)

alpha1, const1 = np.polyfit(logaval1, logprop1, 1)
alpha2, const2 = np.polyfit(logaval2, logprop2, 1)
alpha3, const3 = np.polyfit(logaval3, logprop3, 1)

layout = go.Layout(
    title='Scaling',
    xaxis=dict(title='log Avalanches'),
    yaxis=dict(title='log Probability'),
)

figure = go.Figure([
    go.Scatter(x=logaval1, y=logprop1, mode='markers', name=f'{h1.shape[1]} x {h1.shape[2]}'),
    go.Scatter(x=logaval2, y=logprop2, mode='markers', name=f'{h2.shape[1]} x {h2.shape[2]}'),
    go.Scatter(x=logaval3, y=logprop3, mode='markers', name=f'{h3.shape[1]} x {h3.shape[2]}'),
    go.Scatter(x=logaval1, y=const1 + alpha1 * logaval1, mode='lines', name=f'{h1.shape[1]} x {h1.shape[2]}'),
    go.Scatter(x=logaval2, y=const2 + alpha2 * logaval2, mode='lines', name=f'{h2.shape[1]} x {h2.shape[2]}'),
    go.Scatter(x=logaval3, y=const3 + alpha3 * logaval3, mode='lines', name=f'{h3.shape[1]} x {h3.shape[2]}'),
], layout)

py.iplot(figure)

In [85]:
alpha1, alpha2, alpha3

(-1.6317736795648046, -0.39607881311890136, -2.0524096046159888)

Large number of avalanches (see 100x100) seem to skew the fit a bit. We will redo the fit where we restrict the number of avalanches to $<log(1.5)$.

In [90]:
alpha1, const1 = np.polyfit(logaval1[logaval1 < 1.5], logprop1[logaval1 < 1.5], 1)
alpha2, const2 = np.polyfit(logaval2[logaval2 < 1.5], logprop2[logaval2 < 1.5], 1)
alpha3, const3 = np.polyfit(logaval3[logaval3 < 1.5], logprop3[logaval3 < 1.5], 1)

layout = go.Layout(
    title='Scaling',
    xaxis=dict(title='log Avalanches'),
    yaxis=dict(title='log Probability'),
)

figure = go.Figure([
    go.Scatter(x=logaval1, y=logprop1, mode='markers', name=f'{h1.shape[1]} x {h1.shape[2]}'),
    go.Scatter(x=logaval2, y=logprop2, mode='markers', name=f'{h2.shape[1]} x {h2.shape[2]}'),
    go.Scatter(x=logaval3, y=logprop3, mode='markers', name=f'{h3.shape[1]} x {h3.shape[2]}'),
    go.Scatter(x=logaval1, y=const1 + alpha1 * logaval1, mode='lines', name=f'{h1.shape[1]} x {h1.shape[2]}'),
    go.Scatter(x=logaval2, y=const2 + alpha2 * logaval2, mode='lines', name=f'{h2.shape[1]} x {h2.shape[2]}'),
    go.Scatter(x=logaval3, y=const3 + alpha3 * logaval3, mode='lines', name=f'{h3.shape[1]} x {h3.shape[2]}'),
], layout)

py.iplot(figure)

In [91]:
alpha1, alpha2, alpha3

(-1.011351496446703, -1.209412237557803, -1.9323268773054674)

In [92]:
np.mean([alpha1, alpha2, alpha3])

-1.3843635371033243

In the literature we find $\alpha=1.1$.